In [ ]:
""" Snakemake Script. Don't need to import snakemake."""

import pandas as pd
from os.path import dirname, join
import numpy as np
#import click
#import snakemake
from src.utils.data_io import af_to_vireo
from icecream import ic


In [ ]:
# cells_meta = "/mnt/md0/isshamie/Projects/Mito_Trace/output/pipeline/v03/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/multiplex/cells_meta.tsv"
# sample = ["Input","Control",Flt3l"]
# ref_mt = "/data/Mito_Trace/data/external/chrM_refAllele.txt"

# cov = ["/data/Mito_Trace/output/pipeline/v03/CHIP_b1/MTBlacklist_A2/data/Input/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/Input.coverage.txt",
#        "/data/Mito_Trace/output/pipeline/v03/CHIP_b1/MTBlacklist_A2/data/Control/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/Control.coverage.txt",
#        "/data/Mito_Trace/output/pipeline/v03/CHIP_b1/MTBlacklist_A2/data/Flt3l/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/Flt3l.coverage.txt"]

# output_fs = ["/mnt/md0/isshamie/Projects/Mito_Trace/output/pipeline/v03/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/multiplex/clones_init/donor0",
#             "/mnt/md0/isshamie/Projects/Mito_Trace/output/pipeline/v03/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/multiplex/clones_init/donor1"]

# var_type="init"

#minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20
cov = snakemake.input.cov
output_fs = snakemake.output.af
cells_meta = snakemake.params.cells_meta
sample = snakemake.params.sample
ref_mt = snakemake.params.ref_mt
var_type=snakemake.params.var_type



In [ ]:
cells_meta = pd.read_csv(cells_meta , sep='\t')
#sample = sample.split(',')
ref_fa = pd.read_csv(ref_mt, sep='\t', header=None, index_col=0)

In [ ]:
for d, curr_donor in cells_meta.groupby('donor'):
    d = int(d)
    ic('donor', d)
    vars_conds = []
    af_donor_conds = []
    dp_donor_conds = []

    # Loop through each condition and filter cells for the donor ones
    for ind, curr_cov_f in enumerate(cov): #enumerate(snakemake.input.cov):
        
        ## AF
        af = pd.read_csv(join(dirname(curr_cov_f), "af_by_cell.tsv"), sep='\t',
                         header=0, index_col=0) #cells-by-variants
        condition = sample[ind]
        ic(condition)
        
        af.index = af.index + "_" + condition
#         print('af')
#         print(af.head())
        curr_samp_donor_df = curr_donor[curr_donor["condition"] == condition]  # Condition-donors
        af_donor_samp = af.loc[af.index.isin(curr_samp_donor_df["ID"].values)]
        ic(af_donor_samp.shape)
        af_donor_samp = af_donor_samp.loc[((af_donor_samp > 0).sum(axis=1)>10), ((af_donor_samp > 0).sum(axis=0)>10)] #af_donor_samp.loc[(af_donor_samp.sum(axis=1) > 0), (af_donor_samp.sum(axis=0) > 0)]
        vars_conds.append(set(af_donor_samp.columns))
        af_donor_conds.append(af_donor_samp)
        ic(curr_cov_f)
        ic(af_donor_samp.shape)
        ## Coverage
        curr_samp_cov = pd.read_csv(curr_cov_f,
                                    header=None)  # load sample coverage
        curr_samp_cov[1] = curr_samp_cov[1] + "_" + sample[ind]
        curr_samp_cov = curr_samp_cov.loc[
            curr_samp_cov[1].isin(curr_samp_donor_df["ID"])]
        curr_samp_cov = curr_samp_cov.rename({1: "Cell", 0: "Position", 2: "Value"}, axis=1).pivot(index="Cell",
                                                                                                  columns="Position",
                                                                                                  values="Value")
        dp_donor_samp = pd.DataFrame(index=af_donor_samp.index, columns=af_donor_samp.columns)

        def fill_dp(x,y):
            #print('x', x)
            #print('y', y.head())
            return y.loc[y.index.isin(x.index), int(x.name[:-1])].values

        dp_donor_samp = dp_donor_samp.apply(fill_dp, args=(curr_samp_cov,))

        print('null dp', dp_donor_samp.isnull().sum().sum())
        #dp_donor_samp = dp_donor_samp.loc[dp_donor_samp.index.isin(af_donor_samp.index), dp_donor_samp.columns.isin([int(x[:-1] ) for x in af_donor_samp.columns])]
        dp_donor_conds.append(dp_donor_samp)

    # Get overlap variants and remove cells/vars with 0s
    if var_type == "simpleUnion":
        join_type = "outer"
        overlap_vars = vars_conds[0].union(*vars_conds)
    else:
        join_type="inner"
        overlap_vars = vars_conds[0].intersection(*vars_conds)
    print(join_type)
    #overlap_vars = vars_conds[0].union(*vars_conds)
    print('overlap_vars', len(overlap_vars))
    af_donor_df = pd.concat(af_donor_conds, join=join_type, axis=0,
                                                ignore_index=False)
    print('af_donor_df', af_donor_df.shape)
    af_donor_df = af_donor_df.loc[:, overlap_vars]
    print('af_donor_df after using overlap vars', af_donor_df.shape)
    af_donor_df = af_donor_df.loc[(af_donor_df.sum(axis=1)>0), (af_donor_df.sum(axis=0)>0)]
    print('af_donor_df after 0 removal', af_donor_df.shape)

    #print('af_donor_df')
    #print(af_donor_df.head())
    dp_donor_df = pd.concat(dp_donor_conds, join=join_type, axis=0,
                                                ignore_index=False).fillna(0)
    dp_donor_df = dp_donor_df.loc[dp_donor_df.index.isin(af_donor_df.index), dp_donor_df.columns.isin(af_donor_df.columns)]
    #print('dp_donor_df')

    # Save allele + depth matrix + cells_meta
    af_donor_df.fillna(0).to_csv(join(dirname(output_fs[int(d)]), "af.tsv"),
                       sep='\t', index=True)
    dp_donor_df.fillna(0).to_csv(join(dirname(output_fs[int(d)]), "dp.tsv"),
                       sep='\t', index=True)

    curr_cells_meta = cells_meta[cells_meta["ID"].isin(af_donor_df.index)].copy()
    curr_cells_meta.to_csv(join(dirname(output_fs[int(d)]), "cells_meta.tsv"),
                          sep='\t', index=False)

    ##  Create VireoIn
    
    # Requires AD sparse matrix
    af_donor_df = af_donor_df.fillna(0)
    dp_donor_df = dp_donor_df.fillna(0)
    af_to_vireo(af_donor_df.transpose(), dp_donor_df.transpose(),
                outdir=dirname(output_fs[int(d)]),
                out_name="cellSNP")

    curr_vars = af_donor_df.columns

    af_meta = pd.DataFrame({"POS": [int(x[:-1]) for x in curr_vars],
                                                    "ALT": [x[-1] for x in curr_vars],
                                                    "index": np.arange(1,len(curr_vars)+1),
                                                    "REF": [f"{ref_fa.loc[int(x[:-1]),1]}{x[:-1]}" for x in curr_vars]
                                                }
                                            )
    af_meta["#CHROM"] = "MT"
    af_meta[["#CHROM", "POS", "REF","ALT", "index"]].to_csv(join(dirname(output_fs[d]), "cellSNP.base.vcf"),
                   sep="\t", index=False)


In [ ]:
# print(dp_donor_df.shape)
# dp_donor_df.loc[(dp_donor_df.sum(axis=1)>0), (dp_donor_df.sum(axis=0)>0)].shape
# (dp_donor_df.sum(axis=1) > 1000).sum()

In [ ]:
# print(af_donor_df.shape)
# af_donor_df.loc[(af_donor_df.sum(axis=1)>0), (af_donor_df.sum(axis=0)>0)].shape
# (af_donor_df.sum(axis=1) > 0).sum()